In [1]:
import oommfc as mc
import discretisedfield as df
import micromagneticmodel as mm

region = df.Region(p1=(-50e-9, -50e-9, 0), p2=(50e-9, 50e-9, 10e-9))
mesh = df.Mesh(region=region, cell=(5e-9, 5e-9, 5e-9))

system = mm.System(name='hysteresis')


def Hval(point):
    x, y, z = point
    
    if x < 0:
        return (0, 0, 1e6)
    else:
        return (0, 0, 1e6)

H = df.Field(mesh, dim=3, value=Hval)

system.energy = mm.Exchange(A=1e-12) + mm.Demag()

Ms = 1.1e6

# create system with above geometry and initial magnetisation
system.m = df.Field(mesh, dim=3, value=(0, 0, 1), norm=Ms)

In [2]:
Hmin = (0, 0, -1e6)
Hmax = (0, 0, 1e6)
n = 3

# minimize the energy
hd = mc.HysteresisDriver()
hd.drive(system, Hmin=Hmin, Hmax=Hmax, n=n)

# Plot relaxed configuration: vectors in z-plane
system.m.plane('z').mpl()

Running OOMMF (ExeOOMMFRunner) [2020/12/31 19:01]... (0.4 s)
OOMMF error:
	command: /home/marijanbeg/miniconda3/envs/ud/bin/oommf boxsi +fg hysteresis.mif -exitondone 1
	stdout: /home/marijanbeg/miniconda3/envs/ud/bin/oommf boxsi +fg hysteresis.mif -exitondone 1
	stderr: Logging to file "/home/marijanbeg/miniconda3/envs/ud/opt/oommf/boxsi.errors"

[Boxsi<115796-pupin-marijanbeg> 19:01:05.598 2020-12-31] 2.0a1 infolog:
Start: "/home/marijanbeg/repos/ubermag/oommfc/dev/hysteresis/drive-0/hysteresis.mif"
Options: -exitondone 1 -threads 4

[Boxsi<115796-pupin-marijanbeg> 19:01:05.638 2020-12-31] 2.0a1 LoadProblem error:
Error loading /home/marijanbeg/repos/ubermag/oommfc/dev/hysteresis/drive-0/hysteresis.mif:
	Unknown Event "1"
<115793> oommf.tcl 2.0a1  panic:
child process exited abnormally





RuntimeError: Error in OOMMF run.

In [ ]:
system.table.mpl?

In [ ]:
system.m.orientation((-20e-9, 0, 0))

In [ ]:
system.table

In [ ]:
system.table.mpl(yaxis=['mz'])

In [ ]:
system.table

In [ ]:
import micromagneticmodel as mm

In [ ]:
mm.__version__

In [ ]:
a = ['r1', 'r2']
f'{{main_atlas {" ".join(a)}}}'